# Chain Of Thought (COT)

**What is Chain Of Thought?**

Consider one’s own thought process when solving a complicated reasoning task such as a multi-step math word problem. It is typical to decompose the problem into intermediate steps and solve each before giving the final answer: “After Jane gives 2 flowers to her mom she has 10 . . . then after she gives 3 to her dad she will have 7 . . . so the answer is 7.”

Chain-of-thought prompting has several attractive properties as an approach for facilitating reasoning in language models:

**1.** First, chain of thought, in principle, allows models to decompose multi-step problems into intermediate steps, which means that additional computation can be allocated to problems that require more reasoning steps.

**2.** Second, a chain of thought provides an interpretable window into the behavior of the model, suggesting how it might have arrived at a particular answer and providing opportunities to debug where the reasoning path went wrong (although fully characterizing a model’s computations that support an answer remains an open question).

**3.** Third, chain-of-thought reasoning can be used for tasks such as math word problems, commonsense reasoning, and symbolic manipulation, and is potentially applicable (at least in principle) to any task that humans can solve via language.

**4.** Finally, chain-of-thought reasoning can be readily elicited in sufficiently large off-the-shelf language models simply by including examples of chain of thought sequences into the exemplars of few-shot prompting.

In [ ]:
!pip install python-dotenv
!pip install anthropic
!pip install langchain_openai
!pip install openai
!pip3 install --upgrade typing_extensions

In [7]:
from dotenv import load_dotenv
import anthropic
import random
import os
from langchain_openai import AzureChatOpenAI
import json
from openai import OpenAI
from openai import AzureOpenAI
from pydantic import BaseModel
from openai import OpenAI
import json
import base64
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

True

In [3]:
api_key = os.getenv('AZURE_OPENAI_KEY')
version = os.getenv('AZURE_OPENAI_VERSION')
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
model = "gpt-4o-m"

if api_key is None:
    raise ValueError("API key not found. Please check the environmental variable.")

In [20]:
#print(api_key)
#print(version)
#print(endpoint)

In [14]:
client = AzureOpenAI(
    api_key=api_key,
    api_version=version,
    azure_endpoint=endpoint
    )

In [26]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str
        
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": "Siz yardımcı bir matematik öğretmenisiniz. Yanıtlarınızı LaTeX formatında yazın ve Türkçe açıklamalar ekleyin. Kullanıcıyı çözüm boyunca adım adım yönlendirin. Size matematikle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum diyin."
                       "Kullanıcıya matematiksel ifadeleri açık bir şekilde gösterin ve çözümün her adımını LaTeX formatında verin."},
        {"role": "user", "content": "8x + 7 = -23 nasıl çözülüyor ya?"}
    ],
    response_format=MathReasoning
)

math_reasoning = completion.choices[0].message.parsed

In [27]:
print(math_reasoning)

steps=[Step(explanation="Denklemi çözmeye başlarken, amacımız x'i yalnız bırakmaktır. Bunun için sayısal terimleri ve x terimlerini ayrı ayrı izole etmemiz gerekiyor. İlk adım olarak her iki taraftan da 7 çıkartarak sabit terimlerden kurtulmalıyız.", output='8x + 7 - 7 = -23 - 7'), Step(explanation="Her iki tarafta da 7'ler birbirini götürür, böylece denklem basit bir biçime gelir. Sol tarafta 8x 'i yalnız bırakmış oluruz.", output='8x = -30'), Step(explanation="x'i yalnız bırakmak için 8 ile çarpılıyor. x'in kat sayısını, her iki tarafı 8'e bölerek ortadan kaldırabiliriz.", output='\\frac{8x}{8} = \\frac{-30}{8}'), Step(explanation="8'ler sadeleşir ve sol tarafta sadece x kalır. Sağ tarafta ise -30/8 işlemini yapmamız gerekir. Bu kesir sadeleşebilir.", output='x = \\frac{-30}{8}'), Step(explanation="-30/8 kesirini sadeleştirmek için her iki terimi de 2'ye bölebiliriz.", output='x = \\frac{-15}{4}')] final_answer='x = \\frac{-15}{4}'


In [24]:
type(math_reasoning)

__main__.MathReasoning

In [29]:
print(math_reasoning.final_answer)

x = \frac{-15}{4}


$x = \frac{-15}{4}$

# Verbal Lessons

In [67]:
prompt_verbal = f"""Sen, sözel derslerde tecrübeli bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Her şık üzerinde durarak, neden yanlış olduğunu veya doğru cevaptan nasıl farklı olduğunu detaylıca açıkla. 
Bu açıklamayı, sorunun temelindeki mantığı, metindeki ipuçlarını veya bağlamı analiz ederek yap.
Cevaplama tarzı net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.
"""

In [68]:
print(prompt_verbal)

Sen, sözel derslerde tecrübeli bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Her şık üzerinde durarak, neden yanlış olduğunu veya doğru cevaptan nasıl farklı olduğunu detaylıca açıkla. 
Bu açıklamayı, sorunun temelindeki mantığı, metindeki ipuçlarını veya bağlamı analiz ederek yap.
Cevaplama tarzı net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.



In [48]:
with open('turkce_questions.json') as f:
    turkce_questions_dict = json.load(f, strict=False)

def get_questions_randomly(question_dict, question_keys = None):
    merged_questions = ""
    # if question_keys is None
    if question_keys is None:
        # get random 5 keys from the dictionary
        question_keys = list(question_dict.keys())
        # shuffle the keys
        random.shuffle(question_keys)
        question_keys = question_keys[:5]

    # if question_keys is not None, return the questions as merged string
    for i, key in enumerate(question_keys):
        if key not in question_dict:
            continue
        merged_questions += "Soru " + str(i+1) + ')' + " \n " + question_dict[key] + "\n"

    return merged_questions

def get_questions_from_index(question_dict, start_index, num_questions=1):
    all_keys = list(question_dict.keys())
    
    start_pos = next((i for i, key in enumerate(all_keys) if int(key) > start_index), None)
    
    if start_pos is None:
        raise ValueError("Başlangıç indeksi geçersiz veya sözlükteki anahtarlar içinde bulunamadı.")
    
    end_pos = start_pos + num_questions
    question_keys = all_keys[start_pos:end_pos]
    
    merged_questions = ""
    for i, key in enumerate(question_keys):
        if key not in question_dict:
            continue
        merged_questions += "Soru " + str(i + 1) + ') ' + "\n" + question_dict[key] + "\n"

    return merged_questions

turkce_questions = get_questions_randomly(turkce_questions_dict)

In [ ]:
print(turkce_questions)

In [55]:
class Step(BaseModel):
    explanation: str
    output: str

class TRReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [56]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_verbal},
        {"role": "user", "content": turkce_questions}
    ],
    response_format=TRReasoning
)

tr_answer = completion.choices[0].message.parsed

In [57]:
print(tr_answer)

steps=[Step(explanation='Bu sorunun amacı, paragrafta geçen müzecilikle ilgili bilgilerin hangi başlık altında en iyi şekilde özetleneceğini test etmektir. Paragrafta, müzeciliğin zaman içindeki değişimi ve yeni yaklaşımlar ile birlikte müzeciliğin nasıl bir evrim geçirdiğinden bahsedilmektedir.', output='A) Müzecilikte Yeni Yaklaşımlar'), Step(explanation="'Körelmek' sözcüğü, genellikle işlevini kaybetmek veya etkinliğinin azalması anlamında kullanılır. Seçeneklere bakarsak, 'körelmek' keskinliğini yitirmek, değer, önem veya yeteneğini yitirmek ve ateş veya ışığın sönmesi anlamlarında kullanılabilir. Ancak 'suyu çekilmek' anlamı, organik bir bağlamda, örneğin toprağın kuruması veya bitkinin susuz kalması gibi, körelmekle doğrudan ilişkili değildir.", output='B) Suyu çekilmek'), Step(explanation="Topluluk adı, bir grup insan veya hayvanı ifade eden isimlerdir. Seçeneklerde, 'ordu' topluluk adıdır çünkü birçok askeri ifade eder.", output='D) Dünyanın en büyük ordusu iki kişidir, En kala

In [58]:
print(tr_answer.final_answer)

1) A
2) B
3) D
4) A
5) C


# Numerical Lessons

In [4]:
prompt_numeric = f"""Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Açıklamalarında işlemleri adım adım anlatarak yap.
Cevaplama tarzın net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.
"""

In [5]:
print(prompt_numeric)

Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Açıklamalarında işlemleri adım adım anlatarak yap.
Cevaplama tarzın net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.



In [39]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

soru_tekli = encode_image("tekli_soru.png")
soru_6li = encode_image("6li_soru.png")

In [40]:
class Step(BaseModel):
    explanation: str
    output: str

class MatReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

**Single Question**

In [41]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_numeric},
        {"role": "user", "content": [{"type": "image", "image": soru_tekli}]}
    ],
    response_format=MatReasoning
)

mat_answer = completion.choices[0].message.parsed

In [42]:
print(mat_answer)

steps=[Step(explanation="Öncelikle verilen sayı üzerinde yüzde hesaplaması yapmamız gerekiyor. Yani 75 sayısının %36'sını hesaplayacağız.", output="75 sayısının %36'sını hesaplayalım."), Step(explanation="Yüzde hesaplaması yapmak için sayıyı yüzde oranı ile çarparız ve 100'e böleriz. Yani işlemin matematiksel ifadesi: (75 * 36) / 100 olacaktır.", output='Hesaplanacak ifadeyi belirledik: (75 * 36) / 100'), Step(explanation="Şimdi bu ifadeyi adım adım hesaplayalım. Öncelikle 75 ile 36'yı çarpalım.", output='75 * 36 = 2700'), Step(explanation="Bu çarpma sonucunu 100'e bölersek %36'sını bulmuş oluruz.", output='2700 / 100 = 27'), Step(explanation="Sonuç olarak 75 sayısının %36'sı 27'dir. Bu seçenekler arasında B şıkkına karşılık gelmektedir.", output='Cevap: B) 27')] final_answer='B) 27'


**Multiple Question**

Here, since there is more than one question in the uploaded photo, we have to ask the model to answer all the questions in sequence.

In [53]:
print(prompt_numeric + "Tüm soruları sıra sıra, ayrı ayrı cevapla.")

Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Açıklamalarında işlemleri adım adım anlatarak yap.
Cevaplama tarzın net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.
Tüm soruları sıra sıra, ayrı ayrı cevapla.


In [54]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_numeric + "Tüm soruları sıra sıra, ayrı ayrı cevapla."},
        {"role": "user", "content": [{"type": "image", "image": soru_6li}]}
    ],
    response_format=MatReasoning
)

mat_answer = completion.choices[0].message.parsed

In [55]:
print(mat_answer)

steps=[Step(explanation="75 sayısının %36'sını bulmak için 75'i 0.36 ile çarpmalıyız.", output='75 * 0.36 = 27'), Step(explanation="Bu hesaplamadan, 75 sayısının %36'sının 27 olduğunu bulduk.", output='27')] final_answer='B) 27'


# Irrelevant or Violent Sentences

In [69]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_verbal},
        {"role": "user", "content": "salak mısın? nasıl çözemedin bu soruyu ya"}
    ],
    response_format=TRReasoning
)

answer = completion.choices[0].message.parsed

print(answer.final_answer)

Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum.
